<font size="6"><center>**WHO Data Preparation and Merging for TB Data Analyses**</center></font>

<center>Author Laima Lukoseviciute</center>

# Introduction

I was seeking data on tuberculosis (TB) cases, along with information on each country's population size and income level, to conduct a comprehensive analysis. Unfortunately, I couldn’t locate this specific data on the WHO website, so I decided to devise a solution on my own. This involved a data pre-analysis step, which included merging multiple datasets and transforming them into the format I had envisioned. This notebook serves as the foundation for that preparation, and will be followed by another notebook dedicated to the actual data analysis. 
I obtained the [WHO TB data](https://www.who.int/teams/global-programme-on-tuberculosis-and-lung-health/data) from the WHO website, specifically the `Case notifications [>2Mb]` CSV file. The population data was sourced from the [World Bank](https://data.worldbank.org/indicator/SP.POP.TOTL), while the income classification information for countries was also retrieved from the [World Bank](https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups), extracted from the `Current Classification by Income table in XLSX format`. All relevant files are available in the `data/raw_data` folder.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Importing WHO TB cases dataframe
df_who = pd.read_csv("who_tb_notifications.csv")
print(df_who.shape)
df_who.head()

(9352, 210)


,country,iso2,iso3,iso_numeric,g_whoregion,year,new_sp,new_sn,new_su,new_ep,...,hiv_elig_all_tpt,hiv_elig_all,hiv_elig_new_tpt,hiv_elig_new,hiv_all_tpt,hiv_all,hiv_new_tpt,hiv_new,hiv_all_tpt_completed,hiv_all_tpt_started
0,Afghanistan,AF,AFG,4,EMR,1980,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AF,AFG,4,EMR,1981,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AF,AFG,4,EMR,1982,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AF,AFG,4,EMR,1983,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AF,AFG,4,EMR,1984,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Importing population size dataframe
df_pop = pd.read_csv("world_population.csv")
print(df_pop.shape)
df_pop.head()

(266, 69)


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54922.0,55578.0,56320.0,57002.0,57619.0,58190.0,...,107906.0,108727.0,108735.0,108908.0,109203.0,108587.0,107700.0,107310.0,107359.0,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130072080.0,133534923.0,137171659.0,140945536.0,144904094.0,149033472.0,...,607123269.0,623369401.0,640058741.0,657801085.0,675950189.0,694446100.0,713090928.0,731821393.0,750503764.0,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,9035043.0,9214083.0,9404406.0,9604487.0,9814318.0,10036008.0,...,33831764.0,34700612.0,35688935.0,36743039.0,37856121.0,39068979.0,40000412.0,40578842.0,41454761.0,NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,97630925.0,99706674.0,101854756.0,104089175.0,106388440.0,108772632.0,...,418127845.0,429454743.0,440882906.0,452195915.0,463365429.0,474569351.0,485920997.0,497387180.0,509398589.0,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,5231654.0,5301583.0,5354310.0,5408320.0,5464187.0,5521981.0,...,28157798.0,29183070.0,30234839.0,31297155.0,32375632.0,33451132.0,34532429.0,35635029.0,36749906.0,NaN


In [4]:
# Importing income level dataframe
df_inc = pd.read_csv("countries_income.csv")
print(df_inc.shape)
df_inc.head()

(224, 39)


,Country_code,Country,1987,1988,1989,1990,1991,1992,1993,1994,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AFG,Afghanistan,L,L,L,L,L,L,L,L,...,L,L,L,L,L,L,L,L,L,L
1,ALB,Albania,..,..,..,LM,LM,LM,L,L,...,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM
2,DZA,Algeria,UM,UM,LM,LM,LM,LM,LM,LM,...,UM,UM,UM,UM,UM,LM,LM,LM,LM,UM
3,ASM,American Samoa,H,H,H,UM,UM,UM,UM,UM,...,UM,UM,UM,UM,UM,UM,UM,UM,H,H
4,AND,Andorra,..,..,..,H,H,H,H,H,...,H,H,H,H,H,H,H,H,H,H


In [5]:
# Transforming df_pop
df_pop_long = pd.melt(
    df_pop,
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    var_name="Year",
    value_name="Population"
)

df_pop_long["Year"] = pd.to_numeric(df_pop_long["Year"], errors="coerce")

df_pop_long = df_pop_long.sort_values(by=["Country Name", "Year"]).reset_index(drop=True)
df_pop_long

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Population
0,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1960,9035043.0
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1961,9214083.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1962,9404406.0
3,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1963,9604487.0
4,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1964,9814318.0
...,...,...,...,...,...,...
17285,Zimbabwe,ZWE,"Population, total",SP.POP.TOTL,2020,15526888.0
17286,Zimbabwe,ZWE,"Population, total",SP.POP.TOTL,2021,15797210.0
17287,Zimbabwe,ZWE,"Population, total",SP.POP.TOTL,2022,16069056.0
17288,Zimbabwe,ZWE,"Population, total",SP.POP.TOTL,2023,16340822.0


In [6]:
# Transforming df_inc
df_inc_long = pd.melt(
    df_inc,
    id_vars=["Country_code", "Country"],
    var_name="Year",
    value_name="IncomeGroup"
)

df_inc_long["Year"] = pd.to_numeric(df_inc_long["Year"], errors="coerce")

df_inc_long = df_inc_long.sort_values(by=["Country", "Year"]).reset_index(drop=True)
df_inc_long

,Country_code,Country,Year,IncomeGroup
0,AFG,Afghanistan,1987,L
1,AFG,Afghanistan,1988,L
2,AFG,Afghanistan,1989,L
3,AFG,Afghanistan,1990,L
4,AFG,Afghanistan,1991,L
...,...,...,...,...
8283,ZWE,Zimbabwe,2019,LM
8284,ZWE,Zimbabwe,2020,LM
8285,ZWE,Zimbabwe,2021,LM
8286,ZWE,Zimbabwe,2022,LM


In [7]:
# Merging data sets into one
df_pop_renamed = df_pop_long.rename(columns={
    "Country Code": "iso3",
    "Year": "year",
    "Population": "population_size"
})

df_inc_renamed = df_inc_long.rename(columns={
    "Country_code": "iso3",
    "Year": "year",
    "IncomeGroup": "income_level"
})

df_who = df_who.merge(
    df_pop_renamed[["iso3", "year", "population_size"]],
    on=["iso3", "year"],
    how="left"
)

df_who = df_who.merge(
    df_inc_renamed[["iso3", "year", "income_level"]],
    on=["iso3", "year"],
    how="left"
)
df_who.to_csv("who_tb_data_merged.csv", index=False)
df_who

,country,iso2,iso3,iso_numeric,g_whoregion,year,new_sp,new_sn,new_su,new_ep,...,hiv_elig_new_tpt,hiv_elig_new,hiv_all_tpt,hiv_all,hiv_new_tpt,hiv_new,hiv_all_tpt_completed,hiv_all_tpt_started,population_size,income_level
0,Afghanistan,AF,AFG,4,EMR,1980,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13169311.0,NaN
1,Afghanistan,AF,AFG,4,EMR,1981,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11937581.0,NaN
2,Afghanistan,AF,AFG,4,EMR,1982,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10991378.0,NaN
3,Afghanistan,AF,AFG,4,EMR,1983,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10917982.0,NaN
4,Afghanistan,AF,AFG,4,EMR,1984,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11190221.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9347,Zimbabwe,ZW,ZWE,716,AFR,2019,NaN,NaN,NaN,1957.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15271368.0,LM
9348,Zimbabwe,ZW,ZWE,716,AFR,2020,NaN,NaN,NaN,1140.0,...,39661.0,NaN,NaN,NaN,NaN,NaN,140732.0,156512.0,15526888.0,LM
9349,Zimbabwe,ZW,ZWE,716,AFR,2021,NaN,NaN,NaN,1134.0,...,NaN,NaN,293574.0,1188636.0,54560.0,74234.0,234197.0,293574.0,15797210.0,LM
9350,Zimbabwe,ZW,ZWE,716,AFR,2022,NaN,NaN,NaN,1287.0,...,NaN,NaN,199502.0,1233619.0,49908.0,74693.0,182335.0,201597.0,16069056.0,LM
